# Wordle Helper

A little helper to narrow down your next word guess based on the clues unveiled in previous guesses.

## Adding Google Unigram Frequency List

For this, please download the csv file from [here](https://www.kaggle.com/rtatman/english-word-frequency).

In [ ]:
import csv
from collections import defaultdict

In [ ]:
file = open('unigram_freq.csv')
csvreader = csv.reader(file)

In [ ]:
header = []
header = next(csvreader)
header

In [ ]:
googlewords = defaultdict(int)
for row in csvreader:
        googlewords[row[0]] = int(row[1])

In [ ]:
googlewords['the']

In [ ]:
googlewords5 = dict()
allwords5 = dict()
for word in googlewords.keys():
    if len(word) == 5:
        googlewords5[word] = googlewords[word]
        #if word in brownwords5.keys():
        #    allwords5[word] = googlewords[word] + brownwords5[word]
        allwords5[word] = googlewords[word]

In [ ]:
dict(sorted(allwords5.items(), key=lambda item: item[1], reverse=True))

In [ ]:
len(allwords5)

## Playing Wordle

In [ ]:
elimination = []
black = []
keep = dict()
def play(guess, result):
    newdict = allwords5
    for i in range(5):
        if result[i] == 'b':
            black.append(guess[i])
        if result[i] == 'g':
            keep[guess[i]] = i
        if result[i] == 'y':
            elimination.append((guess[i],i))
    for i in range(5):
        if result[i] == 'g':
            tempdict = newdict
            newdict = dict()
            for word in tempdict.keys():
                if word[i] == guess[i]:
                    elim = 0
                    for g in black:
                        if (g not in keep.keys() and g in word) or (g in keep.keys() and (g in word[:keep[g]] or g in word[keep[g]+1:])):
                        # this handles repeat letters in a guess, where one is green and the other is greyed out
                            elim = 1
                            break
                    if not elim:
                        newdict[word] = allwords5[word]
        if result[i] == 'y':
            tempdict = newdict
            newdict = dict()
            for word in tempdict.keys():
                if guess[i] in word and word[i] != guess[i]:
                    elim = 0
                    for g, e in elimination:
                        if word[e] == g:
                            elim = 1
                            break
                    for g in black:
                        if (g not in keep.keys() and g in word) or (g in keep.keys() and (g in word[:keep[g]] or g in word[keep[g]+1:])):
                        # this handles repeat letters in a guess, where one is yellow and the other is greyed out
                            elim = 1
                            break
                    if not elim:
                        newdict[word] = allwords5[word]
    if result == 'bbbbb':
        tempdict = newdict
        newdict = dict()
        for word in tempdict.keys():
            if not sum([1 if letter in word else 0 for letter in black]):
                newdict[word] = allwords5[word]

    return dict(sorted(newdict.items(), key=lambda item: item[1], reverse=True))

To play one round, use the `play()` function. This function takes in two arguments:
* `guess`: the 5-word you just guessed
* `result`: a 5-character string that describes the colour of the letters in your guess. A green letter is depicted by 'g', a yellow letter is depicted by 'y', and a grey/black letter is depicted with a 'b'.

E.g. If I guess "world" and get the result of "black, green, yellow, yellow, black", I will enter the code:
`play('world','bgyyb')`

When you have selected a word from the resulting list (sorted from highest frequency to lowest) and played it for the next result, you can call the `play()` function again.

In [ ]:
for i in range(6):
    print("Your guess should be a 5-letter word. Enter 'quit' to quit.")
    guess = input("Please input the word you guessed: ").lower()
    if guess == 'quit':
        print("Congratulations, you won with {} tries!".format(i))
        break
    print("Your result should be input as a 5-letter string. Each letter represents the result for each letter.")
    print("'g' for green, 'y' for yellow and 'b' for black (a letter that does not exist in the answer)")
    result = input("Please input your result: ").lower()
    if result == 'ggggg':
        print("Congratulations, you won with {} tries!".format(i))
        break
    nextguess = play(guess, result)
    totalfreq = sum(nextguess.values())
    nextwords = list(nextguess.keys())
    print("Your next guess, in descending order of likelihood (likelihood in brackets): ")
    for x in range(10):
        print(nextwords[x],"{}%".format(round(nextguess[nextwords[x]]/totalfreq*100,2)))

Credits: Inspired by [this post](http://estebanmoro.org/post/2022-01-10-wordle/).